# Implementação do método PrOPPAGA para seleção de ações do Ibovespa

A metodologia Prioridade Observada a Partir da Presunção de Atitude Gaussiana das Alternativas (PrOPPAGA),foi desenvolvida com o objetivo de oferecer uma ferramenta simples para estruturar decisões que envolvem múltiplos critérios.

https://www.proppaga.com.br/

In [5]:
# Bibliotecas Utilizadas
import pandas as pd # Tratamento da base de dados
import matplotlib.pyplot as plt # Visualização dos dados
import seaborn as sns # Visualização dos dados
import math # Calculos matemáticos

# Web scraping https://selenium-python.readthedocs.io/index.html
from selenium import webdriver
from selenium.webdriver.common.by import By

### Tratamento base de dados Ibovespa

In [4]:
# Base de dados de empresas que compõem o ibovespa
# Data de extração: 29/12/2024
# Disponível em: https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/indice-ibovespa-ibovespa-composicao-da-carteira.htm

data_ibov = pd.read_csv('IBOVDia_30-12-24.csv', sep=';')
data_ibov.head()

NameError: name 'pd' is not defined

In [198]:
# Total de empresas que compõem o índice
data_ibov.shape

(87, 6)

In [199]:
# Removendo colunas que não serão utilizadas
data_ibov_clean = data_ibov.drop(columns=data_ibov.columns[[3,4,5]])
data_ibov_clean

,Código,Ação,Tipo
0,ALOS3,ALLOS,ON EJ NM
1,ALPA4,ALPARGATAS,PN N1
2,ABEV3,AMBEV S/A,ON EDJ
3,ASAI3,ASSAI,ON NM
4,AURE3,AUREN,ON NM
...,...,...,...
82,VAMO3,VAMOS,ON NM
83,VBBR3,VIBRA,ON EJ NM
84,VIVA3,VIVARA S.A.,ON NM
85,WEGE3,WEG,ON EJ NM


In [200]:
# Salvando o arquivo para ser utilizado depois
data_ibov_clean.to_csv("lista_ibov.csv", index=False)

### Web Scraping Fundamentus

In [6]:
# Leitura da lista de ativos tratados
df = pd.read_csv('lista_ibov.csv')
df

,Código,Ação,Tipo
0,ALOS3,ALLOS,ON EJ NM
1,ALPA4,ALPARGATAS,PN N1
2,ABEV3,AMBEV S/A,ON EDJ
3,ASAI3,ASSAI,ON NM
4,AURE3,AUREN,ON NM
...,...,...,...
82,VAMO3,VAMOS,ON NM
83,VBBR3,VIBRA,ON EJ NM
84,VIVA3,VIVARA S.A.,ON NM
85,WEGE3,WEG,ON EJ NM


In [7]:
# Inicializa as variáveis que serão extraidas da web
df['setor'] = None
df['LPA'] = None
df['ROA'] = None
df['ROE'] = None
df['P/L'] = None
df['VPA'] = None

In [8]:
df.head()

,Código,Ação,Tipo,setor,LPA,ROA,ROE,P/L,VPA
0,ALOS3,ALLOS,ON EJ NM,None,None,None,None,None,None
1,ALPA4,ALPARGATAS,PN N1,None,None,None,None,None,None
2,ABEV3,AMBEV S/A,ON EDJ,None,None,None,None,None,None
3,ASAI3,ASSAI,ON NM,None,None,None,None,None,None
4,AURE3,AUREN,ON NM,None,None,None,None,None,None


Obter do site https://www.fundamentus.com.br/ as informações para cada ativo.

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# Inicializa o driver especificando o navegador de busca
driver = webdriver.Chrome()

total_lidos = 0
total_erros = 0

# Lista de códigos de ativos que precisam de tratamento especial pois a posição da informação
# está em um estado diferente da pagina do fundamentus
codigos_com_erro = ['BBDC3', 'BBDC4', 'BBAS3', 'BPAC11', 'ITUB4', 'SANB11', 'TIMS3']

for i in df['Código']:
    try:
        # Realiza a consulta no site para cada ativo
        driver.get(f'https://www.fundamentus.com.br/detalhes.php?papel={i}')
        
        # Obtem o valor dos campos para cada classe de busca especificando a posição do objeto
        setor = driver.find_elements(By.CLASS_NAME, 'txt')[13].text
        pl = driver.find_elements(By.CLASS_NAME, 'txt')[32].text
        lpa = driver.find_elements(By.CLASS_NAME, 'txt')[34].text
        vpa = driver.find_elements(By.CLASS_NAME, 'txt')[39].text
        roe = driver.find_elements(By.CLASS_NAME, 'txt')[69].text

        # Inicializa a variavel
        lucro_liquido = None

        # Lógica para pegar o lucro líquido dependendo do código do ativo
        if i in codigos_com_erro:
            lucro_liquido = driver.find_elements(By.CLASS_NAME, 'txt')[106].text
        else:
            lucro_liquido = driver.find_elements(By.CLASS_NAME, 'txt')[110].text
        
        # Converte o valor de lucro líquido em float
        lucro_liquido_float = float(lucro_liquido.replace('.', '').replace(',', '.'))

        # Obtém o valor do ativo e converte em float
        ativo = driver.find_elements(By.CLASS_NAME, 'txt')[87].text.replace('.', '')
        ativo_float = float(ativo)

        # Calcular o ROA (com verificação para divisão por zero)
        roa = None
        if ativo_float != 0.00 and lucro_liquido_float != 0.00:
            roa = round((lucro_liquido_float / ativo_float) * 100, 2)

        # Salva a informação coletada no seu respectivo campo
        df.loc[df['Código'] == i, 'setor'] = setor
        df.loc[df['Código'] == i, 'P/L'] = pl
        df.loc[df['Código'] == i, 'LPA'] = lpa
        df.loc[df['Código'] == i, 'VPA'] = vpa 

        # Existem ativos que não possuem o ROE na plataforma
        if roe != '-':
            df.loc[df['Código'] == i, 'ROE'] = roe
    
        df.loc[df['Código'] == i, 'ROA'] = roa

        total_lidos += 1

    except Exception as e:
        print(f'Erro ao obter os dados para o ativo {i}: {e}')
        total_erros += 1

# Encerra o driver de pesquisa após a extração dos dados
driver.quit()
print(f"total lidos = {total_lidos}")
print(f"total erros = {total_erros}")


total lidos = 87
total erros = 0


In [10]:
# Tipagem e limpeza dos dados coletados
df['ROA'] = df['ROA'].astype('float64')
df['VPA'] = df['VPA'].str.replace(',', '.').astype('float64')
df['P/L'] = df['P/L'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype('float64')
df['LPA'] = df['LPA'].str.replace(',', '.').astype('float64')
df['ROE'] = df['ROE'].str.replace('%', '')
df['ROE'] = df['ROE'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype('float64')

In [11]:
# Verifica se a base possui valores ausentes
df.isnull().sum()

Código    0
Ação      0
Tipo      0
setor     0
LPA       0
ROA       2
ROE       2
P/L       0
VPA       0
dtype: int64

In [24]:
# Identificação dos resgistros ausentes
df[df['ROE'].isnull()]

,Código,Ação,Tipo,setor,LPA,ROA,ROE,P/L,VPA
5,AMOB3,AUTOMOB,ON NM,Automóveis e Motocicletas,0.00,NaN,NaN,0.00,1.23
77,TIMS3,TIM,ON EJ NM,Telecomunicações,0.66,NaN,NaN,21.76,0.00


In [ ]:
# Para a aplicação da metodologia, é nescessário que todos os registros possuam critérios válidos
# Na base de dados coletadas os ativos AMOB3 e TIMS3 não possuem dados válidos de ROA e ROE portanto
# os registros desses ativos serão desconsiderados da base de dados.
df.dropna(inplace=True)

In [26]:
# Estatísticas univariadas
df.describe()

,LPA,ROA,ROE,P/L,VPA
count,85.000000,85.000000,85.000000,85.000000,85.000000
mean,1.768235,4.227412,375.378824,47.586941,15.400706
std,3.483564,9.105556,3354.995313,263.068317,16.352384
min,-13.650000,-34.470000,-71.800000,-47.560000,-74.790000
25%,0.650000,1.430000,5.700000,6.470000,6.690000
50%,1.490000,3.560000,12.700000,8.040000,11.980000
75%,3.300000,7.680000,19.900000,13.640000,22.120000
max,13.650000,44.320000,30942.600000,2374.120000,58.170000
